# Crear y administrar una base de datos no relacional

### Información general
En este tutorial, usted crea una tabla DynamoDB y utiliza la tabla para almacenar y recuperar datos. Utiliza Python y Boto 3, el AWS SDK para Python, para interactuar con las API de DynamoDB. También aprende algunas maneras claves en las que DynamoDB (una base de datos no relacional) difiere de las bases de datos relacionales tradicionales. El tutorial utiliza una aplicación de librería en línea como un ejemplo guía.

### Por qué es importante
Deberá entender cómo modelar sus datos para que se ajusten tanto a los patrones de acceso a datos de su aplicación como al modelo subyacente de su base de datos. Al diseñar una nueva aplicación, desea saber que su elección de base de datos puede manejar la mayor carga proveniente del uso adicional de su aplicación.

DynamoDB proporciona un rendimiento de baja latencia con escalamiento casi infinito, por lo que no necesita preocuparse por los obstáculos de rendimiento a medida que su aplicación crece. Se puede acceder a DynamoDB a través de una API HTTP o un punto de enlace HTTPS, proveyendo un modelo de interacción simple y seguro con su base de datos. Finalmente, DynamoDB es una base de datos NoSQL, que permite un diseño de esquema flexible que puede evolucionar con la aplicación.

In [101]:
import sys
sys.path.append('../')
from decimal import Decimal
from io import BytesIO
from datetime import datetime
import ast
import json
import logging
import os
from pprint import pprint
import requests
from zipfile import ZipFile
import boto3
import pandas as pd
from spdynamodb import DynamoTable
import time

In [162]:
from importlib import reload
import spdynamodb
reload(spdynamodb)
from spdynamodb import DynamoTable

### Información general

Suponga que va a crear una aplicación de una librería en línea. La librería tiene un gran inventario de libros que se almacenan en su almacén para ser vendidos a los clientes. La aplicación debe mostrar a los usuarios los libros que están disponibles para la venta, así como información básica acerca del libro, como el autor.

La aplicación debe recuperar un libro específico en función del título y el autor a fin de que el usuario pueda explorar más detalles cuando vea el libro. Además, la aplicación debe permitir a los usuarios navegar por todos los libros de una categoría específica, como Historia o biografías, para permitir el descubrimiento de libros interesantes.

Finalmente, puede tener diferentes formatos de un libro, como en tapa dura, tapa blanda o audiolibro. La aplicación debe almacenar los formatos de libros en existencia mediante un mapeo asociado a los números de elemento del sistema de inventario. La aplicación debe permitir la actualización de los formatos a través del tiempo, puesto que es posible que se necesiten agregar o eliminar formatos de un libro determinado.

### Terminología
Los siguientes conceptos de DynamoDB son importantes para este módulo:

* **Tabla**: un conjunto de registros de datos de DynamoDB.

* **Elemento**: un solo registro de datos en una tabla de DynamoDB. Es similar a una fila en una base de datos relacional.

* **Atributo**: un único componente de datos que corresponde a un elemento. Es similar a una columna en una base de datos relacional. Sin embargo, a diferencia de las columnas de las bases de datos relacionales, no es necesario especificar los atributos al momento de crear la tabla, salvo la clave principal que se aborda más adelante en este módulo. Los atributos pueden ser de tipo simple, como cadenas, valores enteros o booleanos, así como de tipo complejo, tales como listas o mapas.

* **Clave principal**: una clave principal es un identificador único de un solo elemento en una tabla de DynamoDB. Tanto el nombre de la clave principal como el tipo se deben especificar al momento de crear la tabla. Además, se debe incluir una clave principal del tipo especificado con cada elemento incluido en la tabla. Una clave principal simple está compuesta por un único atributo, mientras que una clave principal compuesta cuenta con dos atributos: una clave de partición y una clave de ordenación. Por ejemplo, puede crear una clave principal simple con “UserID” como identificador o crear una clave principal compuesta con la combinación de “UserID” y “Creation_Date” como identificador de un elemento.

### Modelo de datos

Al momento de crear una aplicación, siempre debe dedicar tiempo al diseño de los modelos de datos necesarios para la lógica de la aplicación. El diseño del modelo de datos debe tener en cuenta las necesidades de acceso a los datos que la aplicación requerirá, tanto para leer como para escribir datos.

DynamoDB es una base de datos no relacional. Si se utilizan bases de datos no relacionales, no es necesario especificar el esquema completo de forma anticipada al crear la tabla. Solo es necesario presentar la clave principal para la tabla, la cual identifica cada registro de la tabla de forma exclusiva. Esto reduce el monto de los costos iniciales correspondientes al diseño del modelo de datos, ya que es posible modificar el esquema fácilmente a medida que cambian las necesidades de la aplicación.

Como se mencionó en la sección “Contexto de la aplicación” de la “Introducción” a este tutorial, la aplicación debe recuperar un libro específico según el título y el autor. Debido a que la combinación del título y el autor representan el identificador único de un libro, puede utilizar esos atributos como la clave principal de la tabla. La aplicación también debe almacenar información sobre la categoría del libro, como historia o biografía, así como información acerca de los formatos disponibles (encuadernación de tapa dura y tapa blanda, o audiolibro). Esa información se asigna a los números del elemento en el sistema que funciona como inventario.

En este contexto, puede utilizar el siguiente esquema para la tabla:

* Título (una cadena): el título del libro
* Autor (una cadena): el autor del libro
* Categoría (una cadena): la categoría a la que pertenece el libro, por ejemplo, historia, biografía y ciencia ficción
* Formatos (un mapa): los diferentes formatos que están disponibles para la venta (por ejemplo, encuadernación de tapa dura y tapa blanda, o audiolibro) y los números del elemento correspondientes en el sistema que funciona como inventario

In [163]:
dt = DynamoTable()

INFO:Found credentials in shared credentials file: ~/.aws/credentials


In [164]:
try:
    dt.select_table('Books')
    print(dt)
except:
    dt.create_table(
        table_name="Books",
        partition_key="Author",
        partition_key_type="S",
        sort_key="Title",
        sort_key_type="S",
        provisioned=False,
    ) 

- Table name: Books            
- Table arn: arn:aws:dynamodb:us-east-1:379442902244:table/Books            
- Table creation: 2023-03-20 15:03:39.657000-03:00            
- [{'AttributeName': 'Author', 'KeyType': 'HASH'}, {'AttributeName': 'Title', 'KeyType': 'RANGE'}]            
- [{'AttributeName': 'Author', 'AttributeType': 'S'}, {'AttributeName': 'Language_code', 'AttributeType': 'S'}, {'AttributeName': 'Title', 'AttributeType': 'S'}]


In [4]:
df = pd.read_csv('books_gr.csv', on_bad_lines='skip')

data_col = list(df.columns)
ref_col = []
for i in data_col:
    if i == "authors":
        i = "Author"
    ref_col.append(i.replace(" ", "").capitalize())
df.columns = ref_col
df.drop_duplicates(subset=["Title", "Author"], inplace=True)

df.head()

,Bookid,Title,Author,Average_rating,Isbn,Isbn13,Language_code,Num_pages,Ratings_count,Text_reviews_count,Publication_date,Publisher
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,9/13/2004,Scholastic


In [7]:
df = df.sample(1000)

In [8]:
dt.batch_pandas(df)

In [9]:
dt.get_item(
    pk_value="Charles Simic",
    se_value="Dime-Store Alchemy: The Art of Joseph Cornell"
)

(None,
 {'ResponseMetadata': {'RequestId': 'BSM5QMRRO8009S4LBM34L9LBBBVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'server': 'Server',
    'date': 'Mon, 20 Mar 2023 18:07:38 GMT',
    'content-type': 'application/x-amz-json-1.0',
    'content-length': '2',
    'connection': 'keep-alive',
    'x-amzn-requestid': 'BSM5QMRRO8009S4LBM34L9LBBBVV4KQNSO5AEMVJF66Q9ASUAAJG',
    'x-amz-crc32': '2745614147'},
   'RetryAttempts': 0}})

Debido a que cada elemento de la tabla se identifica de forma exclusiva con su clave principal, la llamada a la API GetItem siempre devolverá como máximo un elemento de la tabla.

In [65]:
dt.query_items("Susan Cooper", to_pandas=True)

,Num_pages,Isbn13,Publication_date,Isbn,Text_reviews_count,Language_code,Title,Bookid,Ratings_count,Author,Average_rating,Publisher
0,167,9780020421900,4/30/1987,0020421907,72,eng,Seaward,11308,1756,Susan Cooper,3.97,Aladdin Paperbacks
1,274,9780689840333,12/1/2000,0689840330,507,eng,Silver on the Tree (The Dark is Rising #5),11313,27802,Susan Cooper,4.15,Margaret K. McElderry Books


Tanto la llamada a la API GetItem para obtener un único libro como la llamada a la API query_items() para recuperar todos los libros escritos por un autor utilizan la clave primaria especificada en la tabla de Libros. Sin embargo, es posible que desee habilitar patrones de acceso adicionales, como recuperar todos los libros de una categoría, como historia o biografías. La Categoría no hace parte de la clave primaria de la tabla, pero se puede crear un índice secundario a fin de permitir patrones de acceso adicionales.


In [66]:
dt.query_items(
    query="Chaim Potok", 
    to_pandas=True, 
    consumed_capacity=True
)

Consumed Capacity: 0.5


,Num_pages,Isbn13,Publication_date,Isbn,Text_reviews_count,Language_code,Title,Bookid,Ratings_count,Author,Average_rating,Publisher
0,384,9780449001158,9/10/1997,0449001156,339,eng,The Gift of Asher Lev,11502,6224,Chaim Potok,4.16,Ballantine Books


### Crear un índice secundario

DynamoDB permite la creación de índices secundarios para obtener más patrones de acceso a los datos en la tabla. Los índices secundarios son un medio eficaz para agregar flexibilidad de consulta a la tabla de DynamoDB.

DynamoDB tiene dos tipos de índices secundarios: los índices secundarios globales y los índices secundarios locales. En esta sección, se agrega un índice secundario global al atributo de categoría. Este permitirá recuperar todos los libros de una categoría específica.

In [173]:
dt.create_global_secondary_index(att_name="Publisher", att_type="S")

In [174]:
status = dt.check_status_gsi()
time_start = time.time()
while status == 'CREATING':
    status = dt.check_status_gsi()
    time.sleep(30)
total_time = time.time() - time_start
print("Global secondary index created in {} minutes and {} seconds.".format(int(total_time // 60), int(total_time % 60)))

Global secondary index created in 8 minutes and 33 seconds


Crear un índice secundario global es muy parecido a crear una tabla. Se especifican el nombre del índice, los atributos que estarán dentro del índice, el esquema de la clave del índice y el rendimiento aprovisionado (la capacidad máxima que una aplicación puede consumir de una tabla o de un índice). El rendimiento aprovisionado en cada índice es independiente del rendimiento aprovisionado en una tabla. Esto permite definir el rendimiento con detalle a fin de satisfacer las necesidades de la aplicación.

In [ ]:
dt.list_gsi

In [167]:
df = dt.query_items(query="en-GB", index_name="Publisher", to_pandas=True)

In [169]:
df.head()

,Num_pages,Isbn13,Publication_date,Isbn,Text_reviews_count,Language_code,Title,Bookid,Ratings_count,Author,Average_rating,Publisher
0,240,9780974935997,10/1/2005,0974935999,90,en-GB,The Mind Parasites,14495,927,Colin Wilson,3.77,Monkfish Book Publishing
1,542,9780131494848,7/7/2005,0131494848,4,en-GB,Leadership in Organizations,1255,55,Gary Yukl,3.68,Prentice Hall
2,354,9780465008025,6/26/2003,046500802X,78,en-GB,The Evolution Of Desire: Strategies of Human M...,27491,1459,David M. Buss,4.08,Basic Books
3,259,9780898709193,4/30/2006,0898709199,7,en-GB,No Price Too High: A Pentecostal Preacher Beco...,848,51,Alex C. Jones/Diane M. Hanson/Stephen K. Ray,4.27,Ignatius Press
4,328,9780330485388,6/1/2003,0330485385,966,en-GB,The Lovely Bones,537,6485,Alice Sebold,3.81,Picador


In [170]:
df = dt.query_items(query="Simon Schama", to_pandas=True, consumed_capacity=True)

INFO:Consumed Capacity: 0.5


In [171]:
df.head()

,Num_pages,Isbn13,Publication_date,Isbn,Text_reviews_count,Language_code,Title,Bookid,Ratings_count,Author,Average_rating,Publisher
0,825,9780141017273,8/5/2004,0141017279,12,en-GB,Citizens: A Chronicle of the French Revolution,21075,90,Simon Schama,3.99,Penguin


### Actualización de elementos

Además de escribir y leer datos, se busca actualizar regularmente los datos existentes en la base de datos. Es preferible contar con la capacidad de actualizar estos datos mediante una sola llamada a la API, en lugar de primero leer un elemento y luego escribir de regreso el elemento completo con las actualizaciones hechas. DynamoDB permite la actualización de los datos in situ mediante la llamada a la API UpdateItem.

Por ejemplo, recuerde que cada elemento tiene un atributo de Formatos. Este es un mapa de todos los diferentes formatos que la librería tiene de un título específico. Con el tiempo, este mapa se debe actualizar para un libro específico, ya sea porque se ha agregado un nuevo formato o porque ya no se utiliza un formato.

In [172]:
dt

- Table name: Books            
- Table arn: arn:aws:dynamodb:us-east-1:379442902244:table/Books            
- Table creation: 2023-03-20 15:03:39.657000-03:00            
- [{'AttributeName': 'Author', 'KeyType': 'HASH'}, {'AttributeName': 'Title', 'KeyType': 'RANGE'}]            
- [{'AttributeName': 'Author', 'AttributeType': 'S'}, {'AttributeName': 'Language_code', 'AttributeType': 'S'}, {'AttributeName': 'Title', 'AttributeType': 'S'}]